In [25]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

In [26]:
weather_df = pd.read_csv("weather.csv")

In [27]:
useless_columns = [
    "dt", "timezone", "city_name", "lat", "lon", "visibility",
    "sea_level", "rain_3h", "snow_3h", "weather_id", "weather_main",
    "weather_description", "weather_icon", "grnd_level", "clouds_all",
]

# convert dates to strings
weather_df["dt_iso"] = weather_df["dt_iso"].apply(lambda line: line.split()[0])

# process data about precipitation
weather_df.loc[weather_df["rain_1h"].isna(), "rain_1h"] = 0.0
weather_df.loc[weather_df["snow_1h"].isna(), "snow_1h"] = 0.0
weather_df.loc[weather_df["wind_gust"].isna(), "wind_gust"] = 0.0

# drop useless data
weather_df.drop(columns=useless_columns, inplace=True)

In [28]:
# aggregation
weather_df = weather_df.groupby(
    by="dt_iso",
).agg(
    {
        "temp": "mean",
        "dew_point": "mean",
        "feels_like": "mean",
        "temp_min": "min",
        "temp_max": "max",
        "pressure": "mean",
        "humidity": "mean",
        "wind_speed": "mean",
        "wind_deg": "mean",
        "wind_gust": "max", # suddenly increase in weather speed
        "rain_1h": "sum",
        "snow_1h": "sum",
    }
)

In [29]:
# apply standard scaler to it
scaler = StandardScaler()
weather_df.loc[:, :] = scaler.fit_transform(weather_df.values)

In [30]:
weather_df.head()

,temp,dew_point,feels_like,temp_min,temp_max,pressure,humidity,wind_speed,wind_deg,wind_gust,rain_1h,snow_1h
dt_iso,,,,,,,,,,,,
2022-10-01,-0.342081,-0.085892,-0.406331,-0.355213,-0.443872,-0.415206,1.103056,1.942938,0.520756,1.597309,1.583213,-0.190583
2022-10-02,0.059623,0.401451,0.136393,0.434372,-0.297079,0.334515,1.224880,0.984813,1.209440,0.600799,0.638206,-0.190583
2022-10-03,-0.035319,0.153679,0.007835,0.228464,-0.170768,1.098563,0.644997,0.048176,1.621414,-0.304566,-0.569727,-0.190583
2022-10-04,-0.123640,-0.037551,-0.035644,-0.266040,0.188819,0.936203,0.294143,-0.859810,-0.031119,-0.575973,-0.607832,-0.190583
2022-10-05,0.189974,0.333174,0.191051,-0.175246,0.441441,0.654460,0.333127,-0.012075,-0.209667,-0.122278,-0.607832,-0.190583


In [31]:
# read sales data
sales_df = pd.read_csv("daily_data.csv", index_col=0)

In [32]:
sales_df.head()

,Target,Student Price,Worker Price,Guest Price,Student Tax,Worker Tax,Guest Tax,Week,Year,x0_Friday,...,x0_December,x0_February,x0_January,x0_July,x0_June,x0_March,x0_May,x0_November,x0_October,x0_September
2022-01-10,115.0,1.134375,1.17,1.170,0.000000,0.07,0.07,0.037736,0.986341,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2022-01-11,122.0,1.100000,1.17,1.170,0.001284,0.07,0.07,0.037736,0.986341,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2022-01-12,115.0,1.100000,1.17,1.170,0.000000,0.07,0.07,0.037736,0.986341,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2022-01-13,148.0,1.126829,1.17,1.170,0.001138,0.07,0.07,0.037736,0.986341,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2022-01-14,114.0,1.111340,1.17,0.702,0.001443,0.07,0.07,0.037736,0.986341,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [33]:
# join them
merged_df = pd.merge(sales_df, weather_df, left_index=True, right_index=True, how="inner")

In [34]:
merged_df.head()

,Target,Student Price,Worker Price,Guest Price,Student Tax,Worker Tax,Guest Tax,Week,Year,x0_Friday,...,feels_like,temp_min,temp_max,pressure,humidity,wind_speed,wind_deg,wind_gust,rain_1h,snow_1h
2022-10-01,0.0,1.100000,1.267500,1.17,0.0,0.07,0.07,0.735849,0.986341,0.0,...,-0.406331,-0.355213,-0.443872,-0.415206,1.103056,1.942938,0.520756,1.597309,1.583213,-0.190583
2022-10-02,0.0,1.100000,1.267500,1.17,0.0,0.07,0.07,0.735849,0.986341,0.0,...,0.136393,0.434372,-0.297079,0.334515,1.224880,0.984813,1.209440,0.600799,0.638206,-0.190583
2022-10-03,0.0,1.100000,1.267500,1.17,0.0,0.07,0.07,0.754717,0.986341,0.0,...,0.007835,0.228464,-0.170768,1.098563,0.644997,0.048176,1.621414,-0.304566,-0.569727,-0.190583
2022-10-04,102.0,1.246753,1.050000,1.26,0.0,0.07,0.07,0.754717,0.986341,0.0,...,-0.035644,-0.266040,0.188819,0.936203,0.294143,-0.859810,-0.031119,-0.575973,-0.607832,-0.190583
2022-10-05,92.0,1.184000,1.489091,1.26,0.0,0.07,0.07,0.754717,0.986341,0.0,...,0.191051,-0.175246,0.441441,0.654460,0.333127,-0.012075,-0.209667,-0.122278,-0.607832,-0.190583


In [35]:
merged_df.to_csv("extended_data.csv")